In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Load API tokens for our 3rd party APIs.

In [ ]:
from utils import get_circle_api_key,get_gh_api_key,get_vertex_api_key
cci_api_key = get_circle_api_key()
gh_api_key = get_gh_api_key()
vertex_api_key = get_vertex_api_key()